In [1]:
from harvesters.core import Harvester
import numpy as np
from harvesters.util.pfnc import mono_location_formats, rgb_formats, bgr_formats, rgba_formats, bgra_formats

from PIL import Image
import numpy as np

In [2]:
h = Harvester()

In [3]:
h.add_file('D:\AnacondaPython\envs\genicam\Lib\site-packages\genicam\TLSimu.cti');

In [4]:
h.files

['D:\\AnacondaPython\\envs\\genicam\\Lib\\site-packages\\genicam\\TLSimu.cti']

In [5]:
h.update()

In [6]:
for i in h.device_info_list:
    print(i)

(id_='TLSimuMono', vendor='EMVA_D', model='TLSimuMono', tl_type='Custom', user_defined_name='Center', serial_number='SN_InterfaceA_0', version='1.2.3')
(id_='TLSimuColor', vendor='EMVA_D', model='TLSimuColor', tl_type='Custom', user_defined_name='Center', serial_number='SN_InterfaceA_1', version='1.2.3')
(id_='TLSimuMono', vendor='EMVA_D', model='TLSimuMono', tl_type='Custom', user_defined_name='Center', serial_number='SN_InterfaceB_0', version='1.2.3')
(id_='TLSimuColor', vendor='EMVA_D', model='TLSimuColor', tl_type='Custom', user_defined_name='Center', serial_number='SN_InterfaceB_1', version='1.2.3')


In [7]:
ia = h.create_image_acquirer(3)

In [8]:
#ia.remote_device.node_map.Width.value = 8
#ia.remote_device.node_map.Height.value = 8
#ia.remote_device.node_map.PixelFormat.value = 'Mono8'

In [9]:
ia.start_acquisition()

In [10]:
with ia.fetch_buffer() as buffer:
 # Work with the Buffer object. It consists of everything you need.
    print('Buffer: ')
    print(buffer)
    component = buffer.payload.components[0]
    
    width = component.width
    height = component.height
    data_format = component.data_format

    if data_format in mono_location_formats: 
        content = component.data.reshape(height, width) 
    else:
        if data_format in rgb_formats or data_format in rgba_formats or data_format in bgr_formats or data_format in bgra_formats: #
            content = component.data.reshape( height, width, int(component.num_components_per_pixel)) # Set of R, G, B, and Alpha ) #
            if data_format in bgr_formats: # Swap every R and B:
                content = content[:, :, ::-1] 
    print('Data format: ',data_format)
    print('Shape_Content: ',content.shape)
    print('Content: ')
    print(content)
    nodes_map = dir(ia.remote_device.node_map)
    for i in nodes_map:
        print(i)
                                              

Buffer: 
768 x 572, RGB8, 1317888 elements,
[  0   0   0 ...  59 255   0]
Data format:  RGB8
Shape_Content:  (572, 768, 3)
Content: 
[[[  0   0   0]
  [  0   1   0]
  [  0   2   0]
  ...
  [  0 253   0]
  [  0 254   0]
  [  0 255   0]]

 [[  1   0   0]
  [  1   1   0]
  [  1   2   0]
  ...
  [  1 253   0]
  [  1 254   0]
  [  1 255   0]]

 [[  2   0   0]
  [  2   1   0]
  [  2   2   0]
  ...
  [  2 253   0]
  [  2 254   0]
  [  2 255   0]]

 ...

 [[ 57   0   0]
  [ 57   1   0]
  [ 57   2   0]
  ...
  [ 57 253   0]
  [ 57 254   0]
  [ 57 255   0]]

 [[ 58   0   0]
  [ 58   1   0]
  [ 58   2   0]
  ...
  [ 58 253   0]
  [ 58 254   0]
  [ 58 255   0]]

 [[ 59   0   0]
  [ 59   1   0]
  [ 59   2   0]
  ...
  [ 59 253   0]
  [ 59 254   0]
  [ 59 255   0]]]
AcquisitionControl
AcquisitionMode
AcquisitionStart
AcquisitionStop
ChunkDataControl
ChunkModeActive
DeviceControl
DeviceModelName
DeviceSFNCVersionMajor
DeviceSFNCVersionMinor
DeviceSerialNumber
DeviceVendorName
DeviceVersion
EventContr

In [11]:
im = Image.fromarray(content , 'RGB')
im.save('test.bmp')

In [12]:
ia.stop_acquisition()
ia.destroy()

In [13]:
h.reset()